In [1]:
import pandas as pd
import numpy as np

from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""))
# pd.set_option('max_colwidth', 100)

In [2]:
# mf = pd.read_csv('buildstock_MF.csv', index_col='Building')
# sfa = pd.read_csv('buildstock_SFA.csv', index_col='Building')

In [10]:
#Change HVAC options from central systems to unit
buildstock_file = 'buildstock_SFA_nocentral.csv'
mf = pd.read_csv(buildstock_file, index_col='Building')

for idx in mf.index:
    row = mf.loc[idx]
    if row['HVAC System Is Shared'] == 'None':
        continue
    fuel_cols = ['Natural Gas']
    if mf.loc[idx]['HVAC System Shared Natural Gas'] == 'Boiler Baseboards Heating Only':
        mf.loc[idx, 'HVAC System Heating Natural Gas'] = 'Gas Boiler, 80% AFUE'
        mf.loc[idx, 'HVAC System Shared Natural Gas'] = 'None'
        
    elif mf.loc[idx]['HVAC System Shared Natural Gas'] == 'Fan Coil Heating And Cooling':
        mf.loc[idx, 'HVAC System Heating Natural Gas'] = 'Gas Boiler, 80% AFUE'
        mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
        mf.loc[idx, 'HVAC System Shared Natural Gas'] = 'None'
        
    elif mf.loc[idx]['HVAC System Shared Natural Gas'] == 'Fan Coil Cooling Only':
        mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
        mf.loc[idx, 'HVAC System Shared Natural Gas'] = 'None'
        
    elif mf.loc[idx]['HVAC System Shared Natural Gas'] == 'PTAC Heating And Cooling':
        mf.loc[idx, 'HVAC System Heating Natural Gas'] = 'Gas Boiler, 80% AFUE'
        mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
        mf.loc[idx, 'HVAC System Shared Natural Gas'] = 'None'
        
    elif mf.loc[idx]['HVAC System Shared Electricity'] == 'Boiler Baseboards Heating Only':
        mf.loc[idx, 'HVAC System Heating Electricity'] = 'Electric Baseboard'
        mf.loc[idx, 'HVAC System Shared Electricity'] = 'None'
    
    elif mf.loc[idx]['HVAC System Shared Electricity'] == 'Fan Coil Cooling Only': 
        mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
        mf.loc[idx, 'HVAC System Shared Electricity'] = 'None'
        
    elif mf.loc[idx]['HVAC System Shared Electricity'] == 'Fan Coil Heating And Cooling':
        mf.loc[idx, 'HVAC System Cooling'] = 'AC, SEER 10'
        mf.loc[idx, 'HVAC System Heating Electricity'] = 'Electric Baseboard'
        mf.loc[idx, 'HVAC System Shared Electricity'] = 'None'
    
    elif mf.loc[idx]['HVAC System Shared Propane'] == 'Boiler Baseboards Heating Only':
        mf.loc[idx, 'HVAC System Heating Propane'] = 'Propane Boiler, 80% AFUE'
        mf.loc[idx, 'HVAC System Shared Propane'] = 'None'
        
    elif mf.loc[idx]['HVAC System Shared Fuel Oil'] == 'Boiler Baseboards Heating Only':
        mf.loc[idx, 'HVAC System Heating Fuel Oil'] = 'Oil Boiler, 80% AFUE'
        mf.loc[idx, 'HVAC System Shared Fuel Oil'] = 'None'    

In [11]:
mf.to_csv('buildstock_SFA_nocentral.csv')

In [12]:
dirs = ['North', 'Northeast', 'East', 'Southeast', 'South', 'Southwest', 'West', 'Northwest']
inv_dirs = ['South', 'Southwest', 'West', 'Northwest', 'North', 'Northeast', 'East', 'Southeast']
orient_dict = dict(zip(dirs, inv_dirs))

In [13]:
master_df = pd.DataFrame()
buildtype = 'SFA'
df = mf
middle_ct = 0
for idx in df.index:
    bldg = df.loc[idx]
    units = int(bldg[f'Geometry Building Number Units {buildtype}'])
    bstock = pd.DataFrame(columns= df.columns)
    bstock.index.name = 'Building'
    
    if units >= 3 and buildtype == 'SFA':
        mid_mult = units-2
        units = 3
    else:
        mid_mult = 1
    
    if buildtype == 'MF':
        not_type = 'SFA'
        floors = int(bldg['Geometry Stories'])
        num_units_per_floor = units/floors
        front_orient = bldg['Orientation']
        rear_orient = front_orient
        num_middle_z = floors - 2
        if num_units_per_floor%2 != 0: #no rear
            has_rear_units = "No"
            num_middle_x = int(num_units_per_floor - 2)
        else: #rear units
            has_rear_units = "Yes"
            rear_orient = orient_dict[bldg['Orientation']]
            num_middle_x = int((num_units_per_floor/2) - 2)   
        if num_middle_x < 0:
            num_middle_x = 0
        if has_rear_units == 'Yes':
            num_units_front = int(units/2)
            units_add = int(num_units_per_floor/2)
        else:
            num_units_front = int(units)
            units_add = int(num_units_per_floor)
    else:
        not_type = 'MF'
        num_middle_x = units - 2
        has_rear_units = "No"

    #Initial horizontal locations
    h_left = [1]
    if num_middle_x > 0:
        h_mid = list(range(2, 2+int(num_middle_x)))
    else:
        h_mid = []
    if (buildtype=='SFA') or (num_units_front/floors > 1 and buildtype=='MF'):
        h_right = [num_middle_x + 2]
    else:
        h_right = []
        
    #Define horizontal locatons by floor
    if buildtype == 'MF':
        for f in range(2,floors+1):
            h_left += [h_left[-1] + units_add]
            h_mid += list(map(lambda x: x+units_add, h_mid[-num_middle_x:]))
            h_right += list(map(lambda x: x+units_add, h_right[-1:]))

    #LEVELS
    if buildtype == 'MF':
        #..bottom
        if num_units_front/floors > 1:
            lv_bottom = list(range(1,2+num_middle_x+1))
        else:
            lv_bottom = [1]
        #..middle
        if num_middle_z > 0:
            lv_middle = list(range(lv_bottom[-1]+1, lv_bottom[-1]+1 + (units_add*num_middle_z)))
        else:
            lv_middle = []
        #..top
        if floors > 1:
            lv_top = list(range(units_add*(floors-1) + 1, units_add*floors +1))
        else:
            lv_top = []

        if has_rear_units == 'Yes':
            h_left += list(map(lambda x:x+num_units_front, h_left))
            h_mid += list(map(lambda x:x+num_units_front, h_mid))
            h_right += list(map(lambda x:x+num_units_front, h_right))
            lv_bottom += list(map(lambda x:x+num_units_front, lv_bottom))
            lv_middle += list(map(lambda x:x+num_units_front, lv_middle))
            lv_top += list(map(lambda x:x+num_units_front, lv_top))

    floor_unit = 1
   
    for unit in range(1,units+1):     
        bstock.loc[unit] = bldg
        bstock.loc[unit, f'Geometry Building Horizontal Location {not_type}'] = 'None'
        if unit in h_left:
            bstock.loc[unit, f'Geometry Building Horizontal Location {buildtype}'] = 'Left'
        elif unit in h_mid:
            middle_ct += 1
            bstock.loc[unit, f'Geometry Building Horizontal Location {buildtype}'] = 'Middle'
        elif unit in h_right:
            bstock.loc[unit, f'Geometry Building Horizontal Location {buildtype}'] = 'Right'

        if buildtype == 'MF':
            if unit in lv_bottom:
                bstock.loc[unit, 'Geometry Building Level MF'] = 'Bottom'
            elif unit in lv_middle:
                bstock.loc[unit, 'Geometry Building Level MF'] = 'Middle'
            elif unit in lv_top:
                bstock.loc[unit, 'Geometry Building Level MF'] = 'Top'

            if unit <= units/2:
                bstock.loc[unit, 'Orientation'] = front_orient
            else:
                bstock.loc[unit, 'Orientation'] = rear_orient
        else:
            bstock.loc[unit, 'Geometry Building Level MF'] = 'None'
            
    bstock.loc[unit, 'Geometry Building Has Rear Units MF'] = has_rear_units
    bstock['building_code'] = 'bldg_{}'.format(idx)
    bstock['middle_multiplier'] = mid_mult
    
    if master_df.empty:
        master_df = bstock.copy()
    else:
        master_df = master_df.append(bstock)
    
master_df.index= range(1,len(master_df)+1)
master_df.index.rename('Building', inplace=True)

In [20]:
master_df['Geometry Building Horizontal Location SFA'].unique()

array(['Left', 'Middle', 'Right'], dtype=object)

In [16]:
bldg_map = pd.DataFrame(columns=['building_range', 'mid_multiplier'], index=master_df.building_code.unique())
for bldg in master_df.building_code.unique():
    idx = list(master_df[master_df.building_code == bldg].index.values)
    bldg_map.loc[bldg]['building_range'] = '"{}"'.format(str(idx).replace(' ','').strip(']['))
    bldg_map.loc[bldg]['mid_multiplier'] = master_df[master_df.building_code == bldg].iloc[0].middle_multiplier
bldg_map.head()
bldg_map.to_csv(f'./building_id_mapping_{buildtype}.csv')

In [17]:
master_df

,Usage Level,Corridor,Eaves,Door Area,Overhangs,Interior Shading,Doors,Hot Water Fixtures,Solar Hot Water,Ceiling Fan,...,Misc Freezer,Misc Hot Tub Spa,Misc Pool,Misc Pool Heater,Misc Pool Pump,Ducts,Infiltration,Geometry Perimeter Footprint Ratio,building_code,middle_multiplier
Building,,,,,,,,,,,,,,,,,,,,,
1,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,"EF 12, National Average",None,None,None,None,In Finished Space,40 ACH50,0.13,bldg_1,8
2,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,"EF 12, National Average",None,None,None,None,In Finished Space,40 ACH50,0.13,bldg_1,8
3,Low,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,50% Usage,None,Standard Efficiency,...,"EF 12, National Average",None,None,None,None,In Finished Space,40 ACH50,0.13,bldg_1,8
4,Medium,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,100% Usage,None,Standard Efficiency,...,None,None,None,None,None,"20% Leakage, R-4",15 ACH50,0.13,bldg_2,6
5,Medium,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,100% Usage,None,Standard Efficiency,...,None,None,None,None,None,"20% Leakage, R-4",15 ACH50,0.13,bldg_2,6
6,Medium,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,100% Usage,None,Standard Efficiency,...,None,None,None,None,None,"20% Leakage, R-4",15 ACH50,0.13,bldg_2,6
7,High,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,200% Usage,None,Standard Efficiency,...,None,None,None,None,None,"20% Leakage, Uninsulated",40 ACH50,0.13,bldg_3,6
8,High,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,200% Usage,None,Standard Efficiency,...,None,None,None,None,None,"20% Leakage, Uninsulated",40 ACH50,0.13,bldg_3,6
9,High,Double-Loaded Interior,2 ft,20 ft^2,None,"Summer = 0.7, Winter = 0.7",Fiberglass,200% Usage,None,Standard Efficiency,...,None,None,None,None,None,"20% Leakage, Uninsulated",40 ACH50,0.13,bldg_3,6


In [18]:
bldg_map

,building_range,mid_multiplier
bldg_1,"""1,2,3""",8
bldg_2,"""4,5,6""",6
bldg_3,"""7,8,9""",6
bldg_4,"""10,11,12""",14
bldg_5,"""13,14,15""",6
bldg_6,"""16,17,18""",5
bldg_7,"""19,20,21""",14
bldg_8,"""22,23,24""",4
bldg_9,"""25,26,27""",14
bldg_10,"""28,29,30""",28


In [21]:
master_df_save = master_df.drop(['building_code', 'middle_multiplier'], axis=1)
master_df_save.to_csv(f'buildstock_{buildtype}_nocentral_single.csv')